In [1]:
import os
import numpy as np
from PIL import Image
import sqlite3
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# ファイルの拡張子を設定
INCLUDED_EXTENTION = [".png", ".jpg"]
# データベース名
dbname = 'images.db'
# データベースコネクションオブジェクト
conn = sqlite3.connect(dbname)
cur = conn.cursor()
# SQLを実行
cur.execute('DROP TABLE image_info')
cur.execute('CREATE TABLE image_info (id INTEGER PRIMARY KEY AUTOINCREMENT, filename STRING)')
conn.commit()
conn.close()

conn = sqlite3.connect(dbname)
cur = conn.cursor()
# ファイルまでのパス
filenames = sorted(os.listdir('handwriting_pics'))
for filename in filenames:
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    cur.execute('INSERT INTO image_info(filename) values(?)', (filename,))
conn.commit()
cur.close()
conn.close()

conn = sqlite3.connect(dbname)
cur = conn.cursor()
cur.execute('SELECT * FROM image_info')
pics_info = cur.fetchall()
cur.close()
conn.close()

# データの前処理を行うコード
img_test = np.empty((0, 64))

for pic_info in pics_info:
    filename = pic_info[1]
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    img = Image.open(f'handwriting_pics/{filename}').convert('L')
    #画像データ内の最小値が0、最大値が16になるように計算
    img_data256 = 255 - np.array(img.resize((8, 8)))

    min_bright = img_data256.min()
    max_bright = img_data256.max()
    img_data16 = (img_data256 - min_bright) / (max_bright - min_bright) * 16
    #加工した画像データの配列をまとめる
    img_test = np.r_[img_test, img_data16.astype(np.uint8).reshape(1, -1)]

# データの読み込み
digits = load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
logreg = LogisticRegression(max_iter=2000)
logreg_model = logreg.fit(X_train, y_train)

X_true = []
for filename in filenames:
    base, ext = os.path.splitext(filename)
    if ext not in INCLUDED_EXTENTION:
        continue
    X_true = X_true + [int(filename[:1])]
X_true = np.array(X_true)
# 判別する。
pred_logreg = logreg_model.predict(img_test)

print('手書き文字の判別結果')
print('観測結果:', X_true)
print('予測結果:', pred_logreg)
print('正解率:', logreg_model.score(img_test, X_true))

OperationalError: no such table: image_info

In [4]:
dir_name = 'handwriting_pics'

def load_filenames(dir_name, included_ext=INCLUDED_EXTENTION):
    """手書き文字画像が置いてあるパスからファイル名を取得し、リストを作成する関数"""
    files = []
    filenames = sorted(os.listdir(dir_name))
    for filename in filenames:
        base, ext = os.path.splitext(filename)
        if ext not in included_ext:
            continue
        files.append(filename)
    return files

def get_grayscale(dir_name):
    """読み込んだ手書き文字画像の色をグレースケールに変換する関数 (グレースケールは色の濃淡の明暗を分ける技法のことです。)"""
    filenames = load_filenames(dir_name)
    for filename in filenames:
        img = Image.open(f'{dir_name}/{filename}').convert('L')
        yield img

def get_shrinked_img(dir_name):
    """画像サイズを8×8ピクセルのサイズに統一し、明るさも16階調のグレイスケールで白黒に変換する関数"""
    img_test = np.empty((0, 64))
    crop_size = 8
    for img in get_grayscale(dir_name):
        img_data256 = 255 - np.array(img.resize((crop_size, crop_size)))
        min_bright, max_bright = img_data256.min(),  img_data256.max()
        img_data16 = (img_data256 - min_bright) / (max_bright - min_bright) * 16
        img_test = np.r_[img_test, img_data16.astype(np.uint8).reshape(1, -1)]
    return img_test

img_test = get_shrinked_img(dir_name)
get_shrinked_img(dir_name)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  8., 16.,  0.,  0.,  0.,  0.,  0.,
         0., 16., 16.,  8.,  0.,  0.,  0.,  0.,  0.,  8.,  8.,  8.,  0.,
         0.,  0.,  0.,  0.,  8., 16.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., 16.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  3.,
         0.,  0.,  0.,  0.,  0.,  0.,  6.,  3.,  0.,  0.,  0.,  0.,  0.,
         3.,  3.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  3.,  0.,
         0.,  0.,  0.,  3., 16.,  9.,  6.,  0.,  0.,  0.,  

In [5]:
dir_name = 'handwriting_pics'

def load_filenames(dir_name, included_ext=INCLUDED_EXTENTION):
    """手書き文字画像が置いてあるパスからファイル名を取得し、リストを作成"""
    files = []
    filenames = sorted(os.listdir(dir_name))
    for filename in filenames:
        base, ext = os.path.splitext(filename)
        if ext not in included_ext:
            continue
        files.append(filename)
    return files

def create_logreg_model():
    """ロジスティック回帰の学習済みモデルを生成"""
    digits = load_digits()
    X = digits.data
    y = digits.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    logreg = LogisticRegression(max_iter=2000)
    logreg_model = logreg.fit(X_train, y_train)
    return logreg_model

def evaluate_probs(dir_name, img_test, logreg_model):
    """テストデータを利用してロジスティック回帰の学習済みモデルのアウトプットを評価"""
    filenames = load_filenames(dir_name)
    X_true = [int(filename[:1]) for filename in filenames]  
    X_true = np.array(X_true)
    pred_logreg = logreg_model.predict(img_test)
    
    print('手書き文字の判別結果')
    print('観測結果:', X_true)
    print('予測結果:', pred_logreg)
    print('正解率:', logreg_model.score(img_test, X_true))
    return "Propability calculation is successfully finished"

logreg_model = create_logreg_model()
evaluate_probs(dir_name, img_test, logreg_model)

手書き文字の判別結果
観測結果: [0 1 2 3 4 5 6 7 8 9]
予測結果: [4 4 4 4 4 4 4 7 4 4]
正解率: 0.2


'Propability calculation is successfully finished'

In [6]:
# 学習済みモデルを生成
import pickle
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# データを読み込む
digits = load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

logreg = LogisticRegression(max_iter=2000)
# モデルを生成
model = logreg.fit(X_train, y_train)
with open('model.pickle', mode='wb') as fp:
    pickle.dump(model, fp)